In [ ]:
# import pandas as pd

# logs_file_path = 'loginlogs.txt'
# logs_column_names = ['Date', 'Time', 'PC Name', 'User Name']
# df = pd.read_csv(logs_file_path, sep=' ; ', names=logs_column_names, engine='python')

# print(df.head())

In [ ]:
date_format_1 = '%d.%m.%Y'
date_format_2 = '%a %m/%d/%Y'
date_format_3 = '%d-%b-%Y'
date_format_4 = '%d-%b-%y'
date_format_5 = '%d/%m/%Y'
date_format_6 = '%a-%d-%b'

# Create a new column to store the original date values
df['Original_Date'] = df['Date']

# Attempt to parse the 'Date' column using each format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format=date_format_1).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_2)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_3)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_4)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_5)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_6))

# Use boolean indexing to retrieve values from 'Original_Date' where 'Date' is 'NaT'
na_values = df.loc[df['Date'].isna(), 'Original_Date']

print(na_values)

In [6]:
import pandas as pd

# Step 1: Read both files into DataFrames
loginlogs_df = pd.read_csv("loginlogs.txt", sep=" ; ", header=None, names=["date", "time", "PC name", "user name"], engine='python')
school_pcs_df = pd.read_csv("School_PCs.txt", sep=" ; ", header=None, names=["Inventory number", "Asset name", "Asset type", "Location", "Responsible", "Position", "Year", "Comment"], engine='python')

# Step 2: Extract inventory numbers from School_PCs.txt
inventory_numbers = school_pcs_df["Inventory number"]

# Step 3: Filter loginlogs_df to include only rows with PC names containing inventory numbers
df = loginlogs_df[loginlogs_df["PC name"].str.contains('|'.join(map(str, inventory_numbers)), case=False)]

# Display the filtered DataFrame
print(df.head())

              date         time      PC name            user name
6   Fri 09/01/2023  10:31:20.62  011108499NU          abdul.wahab
10  Fri 09/01/2023  10:30:41.34  000553011NU     nazym.alipbayeva
12  Fri 09/01/2023  10:28:11.77  000580520NU  eldar.sharafutdinov
25  Fri 09/01/2023  10:20:22.48  000580519NU       aibek.shokayev
26  Fri 09/01/2023  10:19:15.87  NLT069054NU     kassym.talgatuly


In [7]:
Original_df = df.copy()

In [12]:
df = Original_df.copy()

In [13]:
# Create a new column to store the original date values
df.loc[:,'Original_Date'] = df['date'].copy()


date_format_1 = '%a %m/%d/%Y'
date_format_2 = '%d.%m.%Y'
date_format_3 = '%d/%m/%Y'
date_format_4 = '%d-%b-%y'
date_format_5 = '%Y-%m-%d'


df.loc[:, 'date'] = pd.to_datetime(df['date'], errors='coerce', format=date_format_1).combine_first(
    pd.to_datetime(df['date'], errors='coerce', format=date_format_2)).combine_first(
    pd.to_datetime(df['date'], errors='coerce', format=date_format_3)).combine_first(
    pd.to_datetime(df['date'], errors='coerce', format=date_format_4)).combine_first(
    pd.to_datetime(df['date'], errors='coerce', format=date_format_5))
df

,date,time,PC name,user name,Original_Date
6,2023-09-01 00:00:00,10:31:20.62,011108499NU,abdul.wahab,Fri 09/01/2023
10,2023-09-01 00:00:00,10:30:41.34,000553011NU,nazym.alipbayeva,Fri 09/01/2023
12,2023-09-01 00:00:00,10:28:11.77,000580520NU,eldar.sharafutdinov,Fri 09/01/2023
25,2023-09-01 00:00:00,10:20:22.48,000580519NU,aibek.shokayev,Fri 09/01/2023
26,2023-09-01 00:00:00,10:19:15.87,NLT069054NU,kassym.talgatuly,Fri 09/01/2023
...,...,...,...,...,...
352374,2021-08-16 00:00:00,10:35:43.49,NLT066649NU,kanat.dukenbayev,16-Aug-21
352387,2021-08-16 00:00:00,10:16:42.69,NLT063857NU,tomiris.madiyarova,16/08/2021
352393,2021-08-16 00:00:00,9:59:05.73,NLT042613,nursultan.jyeniskhan,Mon 08/16/2021
352394,2021-08-16 00:00:00,9:58:48.87,011107760NU,saule.mergenbayeva,Mon 08/16/2021


In [15]:
df['date'] = df['date'].fillna(method='ffill')

In [16]:
errors_only = df[df['date'].isna()]
errors_only

,date,time,PC name,user name,Original_Date


In [17]:
date_counts = errors_only['Original_Date'].value_counts(dropna=False)
dict(date_counts)

{}

In [23]:
# Calculate the time differences between consecutive rows
df['time_diff'] = df['date'].diff()

# Find the maximum time difference
max_time_diff = df['time_diff'].max()

# Find the rows corresponding to the maximum time difference
max_time_diff_rows = df[df['time_diff'] == max_time_diff]

# Print the result
print("Maximum Time Difference:", max_time_diff)
print("Rows with Maximum Time Difference:")
print(max_time_diff_rows)

Maximum Time Difference: 15351 days 00:00:00
Rows with Maximum Time Difference:
             date         time      PC name      user name   Original_Date  \
119752 2065-02-23   7:44:00.47  NLT042703NU  almas.turakov  Mon 02/23/2065   
120184 2065-02-23   4:54:24,18  NLT042703NU    aknur.bakyt      23.02.2065   
120196 2065-02-23   4:50:49,46  NLT042703NU    aknur.bakyt      23.02.2065   
120222 2065-02-23   4:41:39,91  NLT042703NU    aknur.bakyt      23.02.2065   

        time_diff  
119752 15351 days  
120184 15351 days  
120196 15351 days  
120222 15351 days  


In [47]:
dict(df['time_diff'][2000:3000])

{7146: Timedelta('0 days 00:00:00'),
 7147: Timedelta('0 days 00:00:00'),
 7148: Timedelta('0 days 00:00:00'),
 7149: Timedelta('0 days 00:00:00'),
 7153: Timedelta('0 days 00:00:00'),
 7158: Timedelta('0 days 00:00:00'),
 7159: Timedelta('0 days 00:00:00'),
 7161: Timedelta('0 days 00:00:00'),
 7164: Timedelta('0 days 00:00:00'),
 7165: Timedelta('0 days 00:00:00'),
 7166: Timedelta('0 days 00:00:00'),
 7168: Timedelta('0 days 00:00:00'),
 7170: Timedelta('0 days 00:00:00'),
 7182: Timedelta('0 days 00:00:00'),
 7187: Timedelta('0 days 00:00:00'),
 7189: Timedelta('0 days 00:00:00'),
 7191: Timedelta('0 days 00:00:00'),
 7192: Timedelta('0 days 00:00:00'),
 7195: Timedelta('0 days 00:00:00'),
 7200: Timedelta('0 days 00:00:00'),
 7201: Timedelta('0 days 00:00:00'),
 7207: Timedelta('0 days 00:00:00'),
 7220: Timedelta('0 days 00:00:00'),
 7235: Timedelta('0 days 00:00:00'),
 7238: Timedelta('0 days 00:00:00'),
 7252: Timedelta('0 days 00:00:00'),
 7259: Timedelta('0 days 00:00:00'),
 

In [5]:
count = len(df['date'])
na_count = df['date'].isna().sum()
print(f'Percentage of NaT is: {round(na_count/count*100,2)}%')

Percentage of NaT is: 0.15%


In [ ]:
df['date'].isna()

In [ ]:
date = pd.NaT
df.loc[df["date"] == date, "Original_Date"].iloc[0]

In [22]:
inv_num = "NLT061083NU"
df.loc[df["PC name"] == inv_num, "date"].iloc[2]

Timestamp('2023-08-31 00:00:00')

In [ ]:
filtered_loginlogs_df[filtered_loginlogs_df["user name"] == "abay.kasken"]

In [ ]:
inv_num = "NLT060945"
school_pcs_df[school_pcs_df["Inventory number"] == inv_num]

In [ ]:
date = pd.Timestamp('2065-02-23 00:00:00')

df[df["date"] == date]

In [ ]:
date_counts = df['date'].value_counts(dropna=False)
dict(date_counts)